# Forest Fire Daily Areas FIltering For Indonesia

In [1]:
import os
import glob
import geopandas as gpd
import pandas as pd  # Ensure pandas is imported
import shutil
from multiprocessing import Pool, cpu_count


Copying all the .shape files in a folder named final_all_shapefiles. 

In [2]:
# Define the relative paths from the script's directory
source_directory = os.path.join("GWIS_2021", "Daily")  # Source directory
destination_directory = os.path.join("GWIS_2021", "Daily", "all_daily")  # Destination directory

# Create the destination directory if it doesn't exist
os.makedirs(destination_directory, exist_ok=True)

# Variable to count total copied files
total_copied_files = 0

# Loop through each folder in the source directory
for folder_name in os.listdir(source_directory):
    folder_path = os.path.join(source_directory, folder_name)

    # Check if this is a directory
    if os.path.isdir(folder_path):
        # Get all files in the current folder
        files = glob.glob(os.path.join(folder_path, '*'))  # Match all files
        
        # Loop through each file and copy it to the destination directory
        for file in files:
            destination_file_path = os.path.join(destination_directory, os.path.basename(file))  # Destination path
            
            # If a file with the same name exists in the destination directory, skip the copying
            if os.path.exists(destination_file_path):
                print(f"Skipping: {file} (Already exists in destination directory)")
                continue
            
            # Copy the file to the destination directory
            shutil.copy(file, destination_directory)
            total_copied_files += 1

# Print a summary message
print(f'Total files copied to the destination: {total_copied_files}')

Skipping: GWIS_2021/Daily/all_daily/Daily_GlobFirev3_GWIS_MCD64A1__2000.prj (Already exists in destination directory)
Skipping: GWIS_2021/Daily/all_daily/Daily_GlobFirev3_GWIS_MCD64A1__2019.dbf (Already exists in destination directory)
Skipping: GWIS_2021/Daily/all_daily/Daily_GlobFirev3_GWIS_MCD64A1__2018.dbf (Already exists in destination directory)
Skipping: GWIS_2021/Daily/all_daily/Daily_GlobFirev3_GWIS_MCD64A1__2001.prj (Already exists in destination directory)
Skipping: GWIS_2021/Daily/all_daily/Daily_GlobFirev3_GWIS_MCD64A1__2003.prj (Already exists in destination directory)
Skipping: GWIS_2021/Daily/all_daily/Daily_GlobFirev3_GWIS_MCD64A1__2017.prj (Already exists in destination directory)
Skipping: GWIS_2021/Daily/all_daily/Daily_GlobFirev3_GWIS_MCD64A1__2016.prj (Already exists in destination directory)
Skipping: GWIS_2021/Daily/all_daily/Daily_GlobFirev3_GWIS_MCD64A1__2002.prj (Already exists in destination directory)
Skipping: GWIS_2021/Daily/all_daily/Daily_GlobFirev3_GWI

Combinng all the shape files and saving. 

In [3]:
# Define relative paths from the script's directory
combined_shapefile_path = os.path.join(destination_directory, 'all_daily.shp')  

# Use glob to find all shapefiles
shapefiles = glob.glob(os.path.join(destination_directory, '*.shp'))

# Create a list to hold the GeoDataFrames
gdf_list = []

# Read each shapefile and append it to the list with filtering
for shapefile in shapefiles:
    gdf = gpd.read_file(shapefile)
    
    # Ensure IDate is in datetime format
    gdf['IDate'] = pd.to_datetime(gdf['IDate'], errors='coerce')  # Convert IDate to datetime

    # Filter based on IDate in the range 2005 to 2015
    filtered_gdf = gdf[(gdf['IDate'] >= '2005-01-01') & (gdf['IDate'] <= '2015-12-31')]
    
    # Append the filtered GeoDataFrame if non-empty
    if not filtered_gdf.empty:
        gdf_list.append(filtered_gdf)

# Combine all GeoDataFrames into one
if gdf_list:  # Check if the list is not empty
    combined_gdf = gpd.GeoDataFrame(pd.concat(gdf_list, ignore_index=True))

    # Save to a new shapefile
    combined_gdf.to_file(combined_shapefile_path)

    print('Successfully combined shapefiles into all_daily_2005_2015.shp')
else:
    print('No observations found in the specified date range.')

: 